In [1]:
from mqt.bench import CompilerSettings, QiskitSettings, get_benchmark
from qiskit import *
import numpy as np

min_qubit = 3 # maximum number of qubit
max_qubit = 5 # minimum number of qubit
gaps = 1
min_qubit = min(max(2, min_qubit), max_qubit) # maximum number of qubit
max_qubit = max(2, max_qubit) # minimum number of qubit
#skip_qubis = max(1, skip_qubits)
gaps = 1
compiler_settings = CompilerSettings(qiskit=QiskitSettings(optimization_level=1))
circuits= []
for i in range(min_qubit, max_qubit, gaps):
    qc = get_benchmark(benchmark_name="qaoa",
                       level="nativegates",
                       circuit_size=i,
                       compiler="qiskit",
                       compiler_settings=compiler_settings,
                       provider_name="ionq",)
    circuits.append((i,qc))
    print("---------------vqe with qubit", i,"---------------")
print("=================================================================================================")
print(circuits)

---------------vqe with qubit 3 ---------------
---------------vqe with qubit 4 ---------------
[(3, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x77166cdaec00>), (4, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7715cb0cc3b0>)]


In [2]:
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector

simulator = AerSimulator()

jobs = []
for qubit_number, circuit_object in circuits:
    # Run noisy simulation (counts)
    job = simulator.run(circuit_object, shots=1000)
    results = job.result()
    counts = results.get_counts()

    # Get ideal distribution from statevector
    qc_nom = circuit_object.remove_final_measurements(inplace=False)
    state = Statevector.from_instruction(qc_nom)
    ideal_probs = state.probabilities_dict()

    jobs.append((qubit_number, counts, ideal_probs))

print("Collected results:")
for res in jobs:
    print(res)


Collected results:
(3, {'010': 135, '101': 177, '011': 169, '100': 175, '110': 172, '001': 172}, {'000': 1.7719891433140784e-10, '001': 0.16666666660760052, '010': 0.16666666660760052, '011': 0.16666666660760057, '100': 0.16666666660760043, '101': 0.16666666660760057, '110': 0.16666666660760057, '111': 1.7719891433224025e-10})
(4, {'1001': 514, '0110': 486}, {'0000': 2.5191300044309198e-17, '0001': 5.368779350009058e-10, '0010': 5.368779350011479e-10, '0011': 2.493003754762642e-09, '0100': 5.368779349976857e-10, '0101': 2.4930037547652307e-09, '0110': 0.4999999928664806, '0111': 5.36877934996029e-10, '1000': 5.368779349988282e-10, '1001': 0.4999999928664806, '1010': 2.4930037547623e-09, '1011': 5.368779349937427e-10, '1100': 2.4930037547625473e-09, '1101': 5.368779349962228e-10, '1110': 5.368779349916915e-10, '1111': 2.5191299846979642e-17})


In [3]:
# Hellinger fidelity function
def hellinger_fidelity_with_expected(p, q):
    """Compute Hellinger fidelity between measured distribution p (counts)
       and expected distribution q (probabilities)."""
    p_sum = sum(p.values())
    p_norm = {k: v / p_sum for k, v in p.items()}

    q_sum = sum(q.values())
    q_norm = {k: v / q_sum for k, v in q.items()}

    overlap = 0.0
    for k in set(p_norm) | set(q_norm):
        overlap += np.sqrt(p_norm.get(k, 0) * q_norm.get(k, 0))
    return overlap**2

In [4]:
for qubit_number, counts, ideal_probs in jobs:
    fidelity = hellinger_fidelity_with_expected(counts, ideal_probs)
    print(f"Hellinger Fidelity (QAOA, {qubit_number} qubits): {fidelity:.4f}")

Hellinger Fidelity (QAOA, 3 qubits): 0.9980
Hellinger Fidelity (QAOA, 4 qubits): 0.9998
